# Hi! welcome to the Fwumious Wabbit workshop

### Before you start, some prerequisites:

This workshop was built and tested on linux and macOS. no guarantees for other operating systems.
if you run into issues, or some of the instructions are outdated, feel free to contact me at ykarni@outbrain.com

You'll need to have [python 3](https://www.python.org/downloads/) installed,
and the up-to-date rust tools (rustc, cargo).

If you don't have rust, we recommend installing with [rustup](https://rustup.rs/).

Create a designated work dir for the workshop.

Download the fwumious wabbit code, and build it:

(make sure to follow these instructions starting from the directory where you run jupyter notebook from,
or use another and just copy the fw binary so that it's available)

```bash
git clone https://github.com/outbrain/fwumious_wabbit.git
cd fwumious_wabbit
cargo build --release
cp target/release/fw .. # if you didn't start from the desired work dir, replace .. with your work dir
cd ..
```

### If you followed these instructions carefully, fwumious wabbit is now ready to run:

In [2]:
!./fw --help

fwumious wabbit 1.5
Andraz Tori <atori@outbrain.com>
Superfast Logistic Regression & Field Aware Factorization Machines

USAGE:
    fw [FLAGS] [OPTIONS]

FLAGS:
        --quiet          Quiet mode, does nothing currently (as we don't output diagnostic data anyway)
    -c, --cache          Use cache file
        --save_resume    save extra state so learning can be resumed later with new data
        --sgd            Disable the Adagrad, normalization and invariant updates
        --adaptive       Use Adagrad
        --noconstant     No intercept
    -t, --testonly       Ignore label information and just test
        --vwcompat       vowpal compatibility mode. Uses slow adagrad, emits warnings for non-compatible features
        --daemon         read data from port 26542
        --foreground     in daemon mode, do not fork and run and run fw process in the foreground
    -h, --help           Prints help information
    -V, --version        Prints version information

OPTIONS:
    -d, --d

### Downloading the dataset
hopefully you already downloaded the dataset files from google drive:

https://drive.google.com/drive/folders/1uNpus6CehoamstYh-JFBE_cwbJ-JLizM?usp=sharing

### Review your working directory

In [28]:
!ls -lh

total 12420496
-rw-r--r--@ 1 ykarni  staff   601B Oct 19 13:32 calc_loss.py
-rw-r--r--@ 1 ykarni  staff   1.4K Oct 19 17:30 create_submission_file.py
-rw-r--r--  1 ykarni  staff   482M Oct 19 12:26 dev.fw.gz
-rw-r--r--  1 ykarni  staff    63M Oct 19 13:13 dev_labels
-rwxr-xr-x  1 ykarni  staff   2.0M Oct 19 13:33 fw
-rw-r--r--@ 1 ykarni  staff    53K Oct 21 12:16 fw_ffm_workshop.ipynb
-rw-r--r--@ 1 ykarni  staff   3.3K Oct 20 13:35 fw_util.py
-rw-r--r--  1 ykarni  staff   156K Oct 19 13:24 sample.fw
-rw-r--r--  1 ykarni  staff   884M Oct 19 17:18 test.fw.gz
-rw-r--r--  1 ykarni  staff   1.9G Oct 19 12:44 train.fw.gz
-rw-r--r--  1 ykarni  staff   2.4G Oct 19 12:57 train_full.fw.gz
-rw-r--r--  1 ykarni  staff   253M Oct 19 13:45 train_labels
-rw-r--r--@ 1 ykarni  staff   261B Oct 19 22:00 vw_namespace_map.csv


### Let's have a glance at our dataset

The dataset is split into train and dev (cross validation), roughly a 80:20 split,

with train.fw.gz containing 69,713,384 records, and dev.fw.gz 17,428,347 records.

let's examine a single record:

In [3]:
!tail -n 1 sample.fw

-1 |A 108f172cd134aa |B 2 |C US>MA>506 |D 1 |E 2383841 |F 11680 |G 869 |H 1903 |I 492733 |J 32678 |K 4304 |L 2412693 |M 1822 |N 373 |O |P 1808 1903 |Q 2 |R 10686485


Let's use the namespace map file to understand better what we see:

In [4]:
!cat vw_namespace_map.csv

A,uuid
B,platform
C,geo_location
D,traffic_source
E,document_id
F,source_id
G,publisher_id
H,categories
I,ad_id
J,campaign_id
K,advertiser_id
L,ad_document_id
M,ad_source_id
N,ad_publisher_id
O,ad_categories
P,user_categories
Q,user_page_views,f32
R,display_id


### Great! now let's give fwumious wabbit a test drive

We'll start by training a simple logistic regression model:

In [5]:
from fw_util import train_loop

max_iterations = 20
print_intermediate_loss = True

In [6]:
common_args_str = " ".join(["--cache", \
    "--linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id", \
    "--linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id", \
    "--linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories"])

optimization_params = "--adaptive --sgd"

model_name = "logistic.1"

In [7]:
iterations = train_loop(common_args_str, optimization_params, model_name, max_iterations, print_intermediate_loss)

train command:
./fw --data train.fw.gz --adaptive --sgd --cache --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories -p logistic.1.train_preds --save_resume -f model.logistic.1

continue training cmd:
./fw --data train.fw.gz --adaptive --sgd --cache --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories -p logistic.1.train_preds --save_resume -f model.logistic.1 -i model.logistic.1

cross-validate command:
./fw --data dev.fw.gz 

### Optional step: kaggle submission
Let's see how would we fare on the Outbrain click prediction kaggle with this very basic model.

In [8]:
from create_submission_file import create_submission_file
from fw_util import create_model_and_predict_for_test_set

create_model_and_predict_for_test_set(common_args_str, optimization_params, model_name, iterations)
create_submission_file("logistic.1.test_preds", "logistic.1.submission.csv")

train command:
./fw --data train_full.fw.gz --adaptive --sgd --cache --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories -p logistic.1.train_preds --save_resume -f model.logistic.1

continue training command:
./fw --data train_full.fw.gz --adaptive --sgd --cache --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories -p logistic.1.train_preds --save_resume -i model.logistic.1

test command:
./fw --data test.fw.gz -i model.logist

We'll drag the output file 'logistic.1.submission.csv' to the target in the Outbrain Kaggle competition "Late Submission" form, which you can find here: https://www.kaggle.com/c/outbrain-click-prediction/data,

Click "Upload" and get the results shortly,

and use the Leaderboard to see where this result would place us

**we scored 0.64318, which would put as at 265th place out of 978. can we do better?**

### Let's try some meta-parameter search:
Try tweaking the learning rate ("-l 0.5") and adagrad smoothing ("--power_t 0.5") command line arguments.
See if you can get better results just by changing them.

Succeeded? great, me too! here's what I have just by trying out a few values:

In [9]:
common_args_str = " ".join(["--cache", \
    "--linear uuid --linear platform --linear geo_location --linear traffic_source", \
    "--linear document_id --linear source_id --linear publisher_id --linear categories", \
    "--linear ad_id --linear campaign_id --linear advertiser_id", \
    "--linear ad_document_id --linear ad_source_id --linear ad_publisher_id", \
    "--linear ad_categories --linear user_categories"])

optimization_params = "--adaptive --sgd --power_t 0.2 -l 0.01"

model_name = "logistic.2"

In [10]:
iterations = train_loop(common_args_str, optimization_params, model_name, max_iterations, print_intermediate_loss)

train command:
./fw --data train.fw.gz --adaptive --sgd --power_t 0.2 -l 0.01 --cache --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories -p logistic.2.train_preds --save_resume -f model.logistic.2

continue training cmd:
./fw --data train.fw.gz --adaptive --sgd --power_t 0.2 -l 0.01 --cache --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories -p logistic.2.train_preds --save_resume -f model.logistic.2 -i model.logistic.2

cr

### That was nice! time for some namespace-mixing action
The big guns! let's try out different feature combinations using the "--linear namespace_a,namespace_b" command line argument.

Go over the namespace list and try to make an educated guess.

How did it go? after some failures, I guessed that combining the publisher_id and advertiser_id might help, and also combining user categories and ad categories, and it did!

In [11]:
common_args_str = " ".join(["--cache", \
    "--linear publisher_id,advertiser_id --linear ad_categories,user_categories", \
    "--linear uuid --linear platform --linear geo_location --linear traffic_source", \
    "--linear document_id --linear source_id --linear publisher_id --linear categories", \
    "--linear ad_id --linear campaign_id --linear advertiser_id", \
    "--linear ad_document_id --linear ad_source_id --linear ad_publisher_id", \
    "--linear ad_categories --linear user_categories"])

optimization_params = "--adaptive --sgd --power_t 0.2 -l 0.01"

model_name = "logistic.3"

In [12]:
iterations = train_loop(common_args_str, optimization_params, model_name, max_iterations, print_intermediate_loss)

train command:
./fw --data train.fw.gz --adaptive --sgd --power_t 0.2 -l 0.01 --cache --linear publisher_id,advertiser_id --linear ad_categories,user_categories --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories -p logistic.3.train_preds --save_resume -f model.logistic.3

continue training cmd:
./fw --data train.fw.gz --adaptive --sgd --power_t 0.2 -l 0.01 --cache --linear publisher_id,advertiser_id --linear ad_categories,user_categories --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --line

### Not bad! but we can do better.

Do we have collisions? try tweaking the hash space size, using the --bit_precision (or -b) command line argument:

In [13]:
common_args_str = " ".join(["--cache -b 25", \
    "--linear publisher_id,advertiser_id --linear ad_categories,user_categories", \
    "--linear uuid --linear platform --linear geo_location --linear traffic_source", \
    "--linear document_id --linear source_id --linear publisher_id --linear categories", \
    "--linear ad_id --linear campaign_id --linear advertiser_id", \
    "--linear ad_document_id --linear ad_source_id --linear ad_publisher_id", \
    "--linear ad_categories --linear user_categories"])

optimization_params = "--adaptive --sgd --power_t 0.2 -l 0.01"

model_name = "logistic.4"

In [14]:
iterations = train_loop(common_args_str, optimization_params, model_name, max_iterations, print_intermediate_loss)

train command:
./fw --data train.fw.gz --adaptive --sgd --power_t 0.2 -l 0.01 --cache -b 25 --linear publisher_id,advertiser_id --linear ad_categories,user_categories --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories -p logistic.4.train_preds --save_resume -f model.logistic.4

continue training cmd:
./fw --data train.fw.gz --adaptive --sgd --power_t 0.2 -l 0.01 --cache -b 25 --linear publisher_id,advertiser_id --linear ad_categories,user_categories --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_sour

### Nice, it doesn't come free though, we pay with model size:

In [15]:
!ls -lh model.* | awk -F " " '{print $5", "$9}'

2.0M, model.logistic.1
2.0M, model.logistic.2
2.0M, model.logistic.3
256M, model.logistic.4


### so, where did logistic regression take us so far?
Time for another kaggle submission:

In [16]:
from create_submission_file import create_submission_file
from fw_util import create_model_and_predict_for_test_set

create_model_and_predict_for_test_set(common_args_str, optimization_params, model_name, iterations)
create_submission_file("logistic.4.test_preds", "logistic.4.submission.csv")

train command:
./fw --data train_full.fw.gz --adaptive --sgd --power_t 0.2 -l 0.01 --cache -b 25 --linear publisher_id,advertiser_id --linear ad_categories,user_categories --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories -p logistic.4.train_preds --save_resume -f model.logistic.4

continue training command:
./fw --data train_full.fw.gz --adaptive --sgd --power_t 0.2 -l 0.01 --cache -b 25 --linear publisher_id,advertiser_id --linear ad_categories,user_categories --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --

**We scored 0.65563, which would place us at 166th place out of 978.** nice improvement of 99 places - logistic regression with feature combinations can go a long way for our use case.

### Sweet! but we want to see some FFM action please...
Let's try to go all-in, and have a field for each namespace:


In [17]:
common_args_str = " ".join(["--cache -b 25 --ffm_k 2 --ffm_bit_precision 25", \
    "--linear publisher_id,advertiser_id --linear ad_categories,user_categories", \
    "--linear uuid --linear platform --linear geo_location --linear traffic_source", \
    "--linear document_id --linear source_id --linear publisher_id --linear categories", \
    "--linear ad_id --linear campaign_id --linear advertiser_id", \
    "--linear ad_document_id --linear ad_source_id --linear ad_publisher_id", \
    "--linear ad_categories --linear user_categories", \
    "--ffm_field_verbose uuid --ffm_field_verbose platform --ffm_field_verbose geo_location", \
    "--ffm_field_verbose traffic_source --ffm_field_verbose document_id", \
    "--ffm_field_verbose source_id --ffm_field_verbose publisher_id", \
    "--ffm_field_verbose categories --ffm_field_verbose ad_id --ffm_field_verbose campaign_id", \
    "--ffm_field_verbose advertiser_id --ffm_field_verbose ad_document_id", \
    "--ffm_field_verbose ad_source_id --ffm_field_verbose ad_publisher_id", \
    "--ffm_field_verbose ad_categories --ffm_field_verbose user_categories"])

optimization_params = "--adaptive --sgd --power_t 0.2 --ffm_power_t 0.2 -l 0.01 --ffm_learning_rate 0.01"

model_name = "ffm.1"

In [18]:
iterations = train_loop(common_args_str, optimization_params, model_name, max_iterations, print_intermediate_loss)

train command:
./fw --data train.fw.gz --adaptive --sgd --power_t 0.2 --ffm_power_t 0.2 -l 0.01 --ffm_learning_rate 0.01 --cache -b 25 --ffm_k 2 --ffm_bit_precision 25 --linear publisher_id,advertiser_id --linear ad_categories,user_categories --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories --ffm_field_verbose uuid --ffm_field_verbose platform --ffm_field_verbose geo_location --ffm_field_verbose traffic_source --ffm_field_verbose document_id --ffm_field_verbose source_id --ffm_field_verbose publisher_id --ffm_field_verbose categories --ffm_field_verbose ad_id --ffm_field_verbose campaign_id --ffm_field_verbose advertiser_id --ffm_field_verbose ad_document_id --ffm_field_verbose ad_source_id --ffm_field

### FFM models are even bigger:

In [19]:
!ls -lh model.* | awk -F " " '{print $5", "$9}'

512M, model.ffm.1
2.0M, model.logistic.1
2.0M, model.logistic.2
2.0M, model.logistic.3
256M, model.logistic.4


### Can we improve further by tweaking the meta parameters?
We sure can, BUT I will leave most of the tweaks for you to experiment with. only change here is using ffm_k (the latent vector length) from 2 to 4 - but consider more tweaks:
* Divide the fields to smaller groups, for example '--ffm_field_verbose uuid,platform,document_id --ffm_field_verbose ad_categories,categories,user_categories'
* Get rid of features in the linear part if they don't help (--interaction blah)
* You can still add more feature combinations!
* Tweak ffm_power_t, ffm_learning_rate for the optimization process


In [20]:
common_args_str = " ".join(["--cache -b 25 --ffm_k 2 --ffm_bit_precision 25", \
    "--linear publisher_id,advertiser_id --linear ad_categories,user_categories", \
    "--linear uuid --linear platform --linear geo_location --linear traffic_source", \
    "--linear document_id --linear source_id --linear publisher_id --linear categories", \
    "--linear ad_id --linear campaign_id --linear advertiser_id", \
    "--linear ad_document_id --linear ad_source_id --linear ad_publisher_id", \
    "--linear ad_categories --linear user_categories", \
    "--ffm_field_verbose uuid --ffm_field_verbose platform --ffm_field_verbose geo_location", \
    "--ffm_field_verbose traffic_source --ffm_field_verbose document_id", \
    "--ffm_field_verbose source_id --ffm_field_verbose publisher_id", \
    "--ffm_field_verbose categories --ffm_field_verbose ad_id --ffm_field_verbose campaign_id", \
    "--ffm_field_verbose advertiser_id --ffm_field_verbose ad_document_id", \
    "--ffm_field_verbose ad_source_id --ffm_field_verbose ad_publisher_id", \
    "--ffm_field_verbose ad_categories --ffm_field_verbose user_categories"])

optimization_params = "--adaptive --sgd --power_t 0.2 --ffm_power_t 0.2 -l 0.01 --ffm_learning_rate 0.01"

model_name = "ffm.2"

In [21]:
iterations = train_loop(common_args_str, optimization_params, model_name, max_iterations, print_intermediate_loss)

train command:
./fw --data train.fw.gz --adaptive --sgd --power_t 0.2 --ffm_power_t 0.2 -l 0.01 --ffm_learning_rate 0.01 --cache -b 25 --ffm_k 2 --ffm_bit_precision 25 --linear publisher_id,advertiser_id --linear ad_categories,user_categories --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories --ffm_field_verbose uuid --ffm_field_verbose platform --ffm_field_verbose geo_location --ffm_field_verbose traffic_source --ffm_field_verbose document_id --ffm_field_verbose source_id --ffm_field_verbose publisher_id --ffm_field_verbose categories --ffm_field_verbose ad_id --ffm_field_verbose campaign_id --ffm_field_verbose advertiser_id --ffm_field_verbose ad_document_id --ffm_field_verbose ad_source_id --ffm_field

### Fresh from the oven: feature binning for numerical features
We haven't touched the numerical feature "user_page_views" yet. in the presentation we saw the numerical feature binning capability in Fwumious Wabbit - let's try it out.

The user_page_views feature has the count of user page views before the display event of the recommendation candidate.
a user may have seen 0, 1, 4, 12, 30 or any old number in between or a bit above that.

We'll use BinnerSqrt with a max value of 10, and resolution=1 - you can tweak those numbers to see if you can get better results.

After defining the new feature (--transform), we can use it either in the linear part, alone or as part of an interaction - or as we do here: as a new field.

Let's see if it will improve our model:

In [22]:
common_args_str = " ".join(["--cache -b 25 --ffm_k 4 --ffm_bit_precision 25", \
    "--transform page_views_sqrt=BinnerSqrt(user_page_views)(10,1)", \
    "--linear publisher_id,advertiser_id --linear ad_categories,user_categories", \
    "--linear uuid --linear platform --linear geo_location --linear traffic_source", \
    "--linear document_id --linear source_id --linear publisher_id --linear categories", \
    "--linear ad_id --linear campaign_id --linear advertiser_id", \
    "--linear ad_document_id --linear ad_source_id --linear ad_publisher_id", \
    "--linear ad_categories --linear user_categories", \
    "--ffm_field_verbose uuid --ffm_field_verbose platform --ffm_field_verbose geo_location", \
    "--ffm_field_verbose traffic_source --ffm_field_verbose document_id", \
    "--ffm_field_verbose source_id --ffm_field_verbose publisher_id", \
    "--ffm_field_verbose categories --ffm_field_verbose ad_id --ffm_field_verbose campaign_id", \
    "--ffm_field_verbose advertiser_id --ffm_field_verbose ad_document_id", \
    "--ffm_field_verbose ad_source_id --ffm_field_verbose ad_publisher_id", \
    "--ffm_field_verbose ad_categories --ffm_field_verbose user_categories --ffm_field_verbose page_views_sqrt"])

optimization_params = "--adaptive --sgd --power_t 0.2 --ffm_power_t 0.2 -l 0.01 --ffm_learning_rate 0.01"

model_name = "ffm.3"

In [23]:
iterations = train_loop(common_args_str, optimization_params, model_name, max_iterations, print_intermediate_loss)

train command:
./fw --data train.fw.gz --adaptive --sgd --power_t 0.2 --ffm_power_t 0.2 -l 0.01 --ffm_learning_rate 0.01 --cache -b 25 --ffm_k 4 --ffm_bit_precision 25 --transform page_views_sqrt=BinnerSqrt(user_page_views)(10,1) --linear publisher_id,advertiser_id --linear ad_categories,user_categories --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories --ffm_field_verbose uuid --ffm_field_verbose platform --ffm_field_verbose geo_location --ffm_field_verbose traffic_source --ffm_field_verbose document_id --ffm_field_verbose source_id --ffm_field_verbose publisher_id --ffm_field_verbose categories --ffm_field_verbose ad_id --ffm_field_verbose campaign_id --ffm_field_verbose advertiser_id --ffm_field_verbo

### To wrap things up, let's see where we are now on Kaggle:

In [24]:
from create_submission_file import create_submission_file
from fw_util import create_model_and_predict_for_test_set

create_model_and_predict_for_test_set(common_args_str, optimization_params, model_name, iterations)

print("creating submission file from predictions")
create_submission_file("ffm.3.test_preds", "ffm.3.submission.csv")
print("all done! good luck.")

train command:
./fw --data train_full.fw.gz --adaptive --sgd --power_t 0.2 --ffm_power_t 0.2 -l 0.01 --ffm_learning_rate 0.01 --cache -b 25 --ffm_k 4 --ffm_bit_precision 25 --transform page_views_sqrt=BinnerSqrt(user_page_views)(10,1) --linear publisher_id,advertiser_id --linear ad_categories,user_categories --linear uuid --linear platform --linear geo_location --linear traffic_source --linear document_id --linear source_id --linear publisher_id --linear categories --linear ad_id --linear campaign_id --linear advertiser_id --linear ad_document_id --linear ad_source_id --linear ad_publisher_id --linear ad_categories --linear user_categories --ffm_field_verbose uuid --ffm_field_verbose platform --ffm_field_verbose geo_location --ffm_field_verbose traffic_source --ffm_field_verbose document_id --ffm_field_verbose source_id --ffm_field_verbose publisher_id --ffm_field_verbose categories --ffm_field_verbose ad_id --ffm_field_verbose campaign_id --ffm_field_verbose advertiser_id --ffm_field_

We scored 0.66619 which would place us on 106th place - 60 places up.

Can you do better? if you want to play in the big league you'll probably need to do some more work on the dataset though.

GOOD LUCK!